In [54]:
import numpy as np
import os
import glob
#Read data
path ="dataset/all_files/*.txt"
files=glob.glob(path)
tweets=""
for file in files:
    f=open(file,'r')
    tweets+=f.read()
    tweets+=('\n')
    

In [55]:
print(tweets)

It 's like that at most public boards .
I wonder if one of the victims opened the door to their apartment for some reason .
Some people believe the Jack O ' Lantern also represents the ancient Celtic warrior 's tradition of taking the heads of their enemies as trophies .
Ready .... GO !!!
Thanks !
com/watch ? v = KrdyCxpB4fs And then the entire Myron Fagan video @ youtube .
This is a video of a band from the Upper Silesian village of Zebowice ( Zembowitz-Fohrendorf ) , it is the town with the highest percentage of Germans in Poland .
I can give you her Skype and she will answer all your questions .
Sørensen , 19 år , er født 8. april 1982 i Kolding .
If i sent you some pic wound use be able to put them up cuz use dont have many also i have lots of links to give use
Why, are you afraid that your son might come and find you drunk in a bar 20 years later and try to kill you or something ...
ALWAYS SPEAK OUT WHEN IN A CLASS WITH A BRAINWASHING TEACHER BECAUSE ONCE YOU SPEAK OTHERS WHO MAY 

In [56]:
from string import punctuation

# get rid of punctuation
tweets = tweets.lower() # lowercase, standardize
all_text = ''.join([c for c in tweets if c not in punctuation])
print(punctuation)
# split by new lines and spaces
tweets_split = all_text.split('\n')
all_text = ' '.join(tweets_split)

# create a list of words
words = all_text.split()


!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


In [57]:
# feel free to use this import 
from collections import Counter

## Build a dictionary that maps words to integers
counts = Counter(words)
vocab = sorted(counts, key=counts.get, reverse=True)
vocab_to_int = {word: ii for ii, word in enumerate(vocab, 1)}

## use the dict to tokenize each review in reviews_split
## store the tokenized reviews in reviews_ints
tweets_ints = []
for tweet in tweets_split:
    tweets_ints.append([vocab_to_int[word] for word in tweet.split()])

In [58]:
# stats about vocabulary
print('Unique words: ', len((vocab_to_int)))  # should ~ 74000+
print()

# print tokens in first review
print('Tokenized review: \n', tweets_ints[:1])

Unique words:  16749

Tokenized review: 
 [[9, 19, 28, 10, 37, 138, 303, 3911]]


In [59]:
import pandas as pd

with open('dataset/labels.txt', 'r') as f:
    labels = f.read()

In [60]:
print(type(labels))

<class 'str'>


In [61]:
labels_split=labels.split('\n')
encoded_labels=[]
for label in labels_split:
    if label=='noHate':
        encoded_labels.append(0)
    elif label=='hate':
        encoded_labels.append(1)
    elif label=='relation':
        encoded_labels.append(2)
    elif label=='idk/skip':
        encoded_labels.append(3)


In [388]:
print(len(tweets_ints))
print(set(encoded_labels))

10937
{0, 1, 2, 3}


In [63]:
tweet_lens = Counter([len(x) for x in tweets_ints])
print("Zero-length tweets: {}".format(tweet_lens[0]))
print("Maximum tweet length: {}".format(max(tweet_lens)))

Zero-length tweets: 8
Maximum tweet length: 326


In [64]:
print('Number of tweets before removing outliers: ', len(tweets_ints))

## remove any reviews/labels with zero length from the reviews_ints list.

# get indices of any reviews with length 0
non_zero_idx = [ii for ii, tweet in enumerate(tweets_ints) if len(tweet) != 0]

# remove 0-length reviews and their labels
tweets_ints = [tweets_ints[ii] for ii in non_zero_idx]
encoded_labels = np.array([encoded_labels[ii] for ii in non_zero_idx])

print('Number of reviews after removing outliers: ', len(tweets_ints))

Number of tweets before removing outliers:  10945
Number of reviews after removing outliers:  10937


In [65]:
def pad_features(tweets_ints, seq_length):
    ''' Return features of review_ints, where each review is padded with 0's 
        or truncated to the input seq_length.
    '''
    
    # getting the correct rows x cols shape
    features = np.zeros((len(tweets_ints), seq_length), dtype=int)

    # for each review, I grab that review and 
    for i, row in enumerate(tweets_ints):
        features[i, -len(row):] = np.array(row)[:seq_length]
    
    return features

In [170]:
seq_length = 30

features = pad_features(tweets_ints, seq_length=seq_length)

## test statements - do not change - ##
assert len(features)==len(tweets_ints), "Your features should have as many rows as reviews."
assert len(features[0])==seq_length, "Each feature row should contain seq_length values."

# print first 10 values of the first 30 batches 
print(features[:10,10:])

[[   0    0    0    0    0    0    0    0    0    0    0    0    9   19
    28   10   37  138  303 3911]
 [   0    0    0    0    3  255   31   49    6    1  785 1245    1  742
     2   34 1246   12   60  390]
 [1626  925 7264  132 2752    1 1514  962 2753   19 2401    6  602    1
  1023    6   34  834   26 5004]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0 1024   70]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0  240]
 [   0    0    0    0    0    0    0    0    0 2754  502 7265    4   78
     1 1126 7266 7267  128   52]
 [  39    1 1409 3912 1515    6 5005 5006    9    8    1  223   20    1
  3216 2402    6 1127    7  876]
 [   0    0    0    0    0    0    0    3   50  225   11   81 7268    4
   113   38 1076   23   41 1323]
 [   0    0    0    0    0    0    0    0    0    0 2403 1324 7269 1516
  7270  809 2181 5007    3 5008]
 [  24  391    2  164   44   47 3914  194  127   14  10

In [171]:
split_frac = 0.8

## split data into training, validation, and test data (features and labels, x and y)

split_idx = int(len(features)*split_frac)
train_x, remaining_x = features[:split_idx], features[split_idx:]
train_y, remaining_y = encoded_labels[:split_idx], encoded_labels[split_idx:]

test_idx = int(len(remaining_x)*0.5)
val_x, test_x = remaining_x[:test_idx], remaining_x[test_idx:]
val_y, test_y = remaining_y[:test_idx], remaining_y[test_idx:]

## print out the shapes of your resultant feature data
print("\t\t\tFeature Shapes:")
print("Train set: \t\t{}".format(train_x.shape), 
      "\nValidation set: \t{}".format(val_x.shape),
      "\nTest set: \t\t{}".format(test_x.shape))

			Feature Shapes:
Train set: 		(8749, 30) 
Validation set: 	(1094, 30) 
Test set: 		(1094, 30)


In [204]:
import torch
from torch.utils.data import TensorDataset, DataLoader

# create Tensor datasets
train_data = TensorDataset(torch.from_numpy(train_x), torch.from_numpy(train_y))
valid_data = TensorDataset(torch.from_numpy(val_x), torch.from_numpy(val_y))
test_data = TensorDataset(torch.from_numpy(test_x), torch.from_numpy(test_y))

# dataloaders
batch_size = 50

# make sure the SHUFFLE your training data
train_loader = DataLoader(train_data, shuffle=True,drop_last=True, batch_size=batch_size)
valid_loader = DataLoader(valid_data, shuffle=True,drop_last=True, batch_size=batch_size)
test_loader = DataLoader(test_data, shuffle=True,drop_last=True, batch_size=batch_size)

In [205]:
# obtain one batch of training data
dataiter = iter(train_loader)
sample_x, sample_y = dataiter.next()

print('Sample input size: ', sample_x.size()) # batch_size, seq_length
print('Sample input: \n', sample_x)
print()
print('Sample label size: ', sample_y.size()) # batch_size
print('Sample label: \n', sample_y)

Sample input size:  torch.Size([50, 30])
Sample input: 
 tensor([[    0,     0,     0,  ...,   176,  2042,  7822],
        [    0,     0,     0,  ..., 10375,  3607,   945],
        [    0,     0,     0,  ...,     6,     1,  1503],
        ...,
        [   22,   410,    21,  ...,   316,     6,   109],
        [    0,     0,     0,  ...,   222, 11528,   185],
        [    0,     0,     0,  ...,   118,    32,  1442]])

Sample label size:  torch.Size([50])
Sample label: 
 tensor([0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 2, 1, 0, 0, 0, 0, 0, 2, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0,
        0, 0])


In [206]:
train_on_gpu=torch.cuda.is_available()

if(train_on_gpu):
    print('Training on GPU.')
else:
    print('No GPU available, training on CPU.')

No GPU available, training on CPU.


In [371]:
import torch.nn as nn

class SentimentRNN(nn.Module):
    """
    The RNN model that will be used to perform Sentiment analysis.
    """

    def __init__(self, vocab_size, output_size, embedding_dim, hidden_dim, n_layers, drop_prob=0.5):
        """
        Initialize the model by setting up the layers.
        """
        super(SentimentRNN, self).__init__()

        self.output_size = output_size
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim
        
        # embedding and LSTM layers
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, n_layers, 
                            dropout=drop_prob, batch_first=True)
        
        # dropout layer
        self.dropout = nn.Dropout(0.3)
        
        # linear and sigmoid layers
        self.fc = nn.Linear(hidden_dim, output_size)
        self.sig = nn.Softmax(dim=1)
        

    def forward(self, x, hidden):
        """
        Perform a forward pass of our model on some input and hidden state.
        """
        batch_size = x.size(0)

        # embeddings and lstm_out
        x = x.long()
        embeds = self.embedding(x)
        lstm_out, hidden = self.lstm(embeds, hidden)
    
        # stack up lstm outputs
        lstm_out = lstm_out.contiguous().view(-1, self.hidden_dim)
        
        # dropout and fully-connected layer
        out = self.dropout(lstm_out)
        out = self.fc(out)
        # sigmoid function
        sig_out = self.sig(out)
        
        # reshape to be batch_size first
        sig_out = sig_out.view(batch_size, -1)
        sig_out = sig_out[:, -1] # get last batch of labels
        
        # return last sigmoid output and hidden state
        return sig_out, hidden
    
    
    def init_hidden(self, batch_size):
        ''' Initializes hidden state '''
        # Create two new tensors with sizes n_layers x batch_size x hidden_dim,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
        
        if (train_on_gpu):
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda(),
                  weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_(),
                      weight.new(self.n_layers, batch_size, self.hidden_dim).zero_())
        
        return hidden
        

In [391]:
# Instantiate the model w/ hyperparams
vocab_size = len(vocab_to_int)+1 # +1 for the 0 padding + our word tokens
output_size = 4
embedding_dim = 400
hidden_dim = 256
n_layers = 2

net = SentimentRNN(vocab_size, output_size, embedding_dim, hidden_dim, n_layers)

print(net)


SentimentRNN(
  (embedding): Embedding(16750, 400)
  (lstm): LSTM(400, 256, num_layers=2, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.3, inplace=False)
  (fc): Linear(in_features=256, out_features=4, bias=True)
  (sig): Softmax(dim=1)
)


In [392]:
# loss and optimization functions
lr=0.01

criterion = nn.MSELoss()
optimizer = torch.optim.Adam(net.parameters(), lr=lr)


In [393]:
# training params

epochs = 4 # 3-4 is approx where I noticed the validation loss stop decreasing

counter = 0
print_every = 100
clip=5 # gradient clipping

# move model to GPU, if available
if(train_on_gpu):
    net.cuda()

net.train()
# train for some number of epochs
for e in range(epochs):
    # initialize hidden state
    h = net.init_hidden(batch_size)

    # batch loop
    for inputs, labels in train_loader:
        counter += 1

        if(train_on_gpu):
            inputs, labels = inputs.cuda(), labels.cuda()

        # Creating new variables for the hidden state, otherwise
        # we'd backprop through the entire training history
        h = tuple([each.data for each in h])

        # zero accumulated gradients
        net.zero_grad()

        # get the output from the model
        output, h = net(inputs, h)

        # calculate the loss and perform backprop
        loss = criterion(output.squeeze(), labels.float())
        loss.backward()
        # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
        nn.utils.clip_grad_norm_(net.parameters(), clip)
        optimizer.step()

        # loss stats
        if counter % print_every == 0:
            # Get validation loss
            val_h = net.init_hidden(batch_size)
            val_losses = []
            net.eval()
            for inputs, labels in valid_loader:

                # Creating new variables for the hidden state, otherwise
                # we'd backprop through the entire training history
                val_h = tuple([each.data for each in val_h])

                if(train_on_gpu):
                    inputs, labels = inputs.cuda(), labels.cuda()

                output, val_h = net(inputs, val_h)
                val_loss = criterion(output.squeeze(), labels.float())

                val_losses.append(val_loss.item())

            net.train()
            print("Epoch: {}/{}...".format(e+1, epochs),
                  "Step: {}...".format(counter),
                  "Loss: {:.6f}...".format(loss.item()),
                  "Val Loss: {:.6f}".format(np.mean(val_losses)))

Epoch: 1/4... Step: 100... Loss: 0.331687... Val Loss: 0.263858
Epoch: 2/4... Step: 200... Loss: 0.303547... Val Loss: 0.275609
Epoch: 2/4... Step: 300... Loss: 0.174639... Val Loss: 0.270778
Epoch: 3/4... Step: 400... Loss: 0.499921... Val Loss: 0.302079
Epoch: 3/4... Step: 500... Loss: 0.428224... Val Loss: 0.287516
Epoch: 4/4... Step: 600... Loss: 0.099828... Val Loss: 0.297552


In [394]:
print(inputs.size())
print(len(val_h), val_h[0].shape,val_h[1].shape)
print(hidden_dim)

torch.Size([50, 30])
2 torch.Size([2, 50, 256]) torch.Size([2, 50, 256])
256


In [395]:
# Get test data loss and accuracy

test_losses = [] # track loss
num_correct = 0

# init hidden state
h = net.init_hidden(batch_size)

net.eval()
# iterate over test data
for inputs, labels in test_loader:

    # Creating new variables for the hidden state, otherwise
    # we'd backprop through the entire training history
    h = tuple([each.data for each in h])

    if(train_on_gpu):
        inputs, labels = inputs.cuda(), labels.cuda()
    
    # get predicted outputs
    output, h = net(inputs, h)
    
    # calculate loss
    test_loss = criterion(output.squeeze(), labels.float())
    test_losses.append(test_loss.item())
    
    # convert output probabilities to predicted class (0 or 1)
    pred = torch.round(output.squeeze())  # rounds to the nearest integer
    
    # compare predictions to true label
    correct_tensor = pred.eq(labels.float().view_as(pred))
    correct = np.squeeze(correct_tensor.numpy()) if not train_on_gpu else np.squeeze(correct_tensor.cpu().numpy())
    num_correct += np.sum(correct)


# -- stats! -- ##
# avg test loss
print("Test loss: {:.3f}".format(np.mean(test_losses)))

# accuracy over all test data
test_acc = num_correct/len(test_loader.dataset)
print("Test accuracy: {:.3f}".format(test_acc))

Test loss: 0.303
Test accuracy: 0.753


In [396]:
# negative test review
test_review_neg = 'Trump is a dumb bitch'


In [397]:
from string import punctuation

def tokenize_review(test_review):
    test_review = test_review.lower() # lowercase
    # get rid of punctuation
    test_text = ''.join([c for c in test_review if c not in punctuation])

    # splitting by spaces
    test_words = test_text.split()

    # tokens
    test_ints = []
    test_ints.append([vocab_to_int[word] for word in test_words])

    return test_ints

# test code and generate tokenized review
test_ints = tokenize_review(test_review_neg)
print(test_ints)

[[3654, 8, 5, 629, 2452]]


In [398]:
# test sequence padding
seq_length=200
features = pad_features(test_ints, seq_length)

print(features)

[[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0   

In [399]:
# test conversion to tensor and pass into your model
feature_tensor = torch.from_numpy(features)
print(feature_tensor.size())

torch.Size([1, 200])


In [400]:
def predict(net, test_review, sequence_length=200):
    
    net.eval()
    
    # tokenize review
    test_ints = tokenize_review(test_review)
    
    # pad tokenized sequence
    seq_length=sequence_length
    features = pad_features(test_ints, seq_length)
    
    # convert to tensor to pass into your model
    feature_tensor = torch.from_numpy(features)
    
    batch_size = feature_tensor.size(0)
    
    # initialize hidden state
    h = net.init_hidden(batch_size)
    
    if(train_on_gpu):
        feature_tensor = feature_tensor.cuda()
    
    # get the output from the model
    output, h = net(feature_tensor, h)
    
    # convert output probabilities to predicted class (0 or 1)
    pred = torch.round(output.squeeze()) 
    # printing output value, before rounding
    print('Prediction value, pre-rounding: {:.6f}'.format(output.item()))
    
    # print custom response
    if(pred.item()==1):
        print("Hate speech detected!")
    elif(pred.item()==2):
        print("Relation detected.")
    elif(pred.item()==3):
        print("Idk")
    elif(pred.item()==0):
        print("No hate speech detected!")
        

In [410]:
test_tweet="I hate hate hate hate hate black people"
seq_length=200

predict(net, test_tweet, seq_length)

Prediction value, pre-rounding: 0.099398
No hate speech detected!


In [314]:
#Test un autre modèle, ne marche pas
import torch.nn as nn
import torch.nn.functional as F

class CNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, n_filters, filters, filter_sizes, output_dim, dropout,
                pad_idx):
        super().__init__()
        self.embedding=nn.Embedding(vocab_size, embedding_dim)
        self.convs=nn.ModuleList([
            nn.Conv2d(in_channels=1,
                     out_channels=n_filters,
                     kernel_size=(fs,embedding_dim))
            for fc in filter_sizes
        ])
        
        self.fc=nn.Linear(len(filter_sizes)*n_filters, output_dim)
        self.dropout=nn.Dropout(dropout)

    def forward(self,text):
        text=text.permute(1,0)
        embedded= self.embedding(text)
        embedded=embedded.unsqueeze(1)
        
        conved=[F.Relu(conv(embedded)).squeeze(3) for conv in self.convs]
        
        pooled=[F.max_pool1d(conv,conv.shape[2]).squeeze(2) for conv in conved]
        
        cat = self.dropout(torch.cat(pooled, dim=1))
        
        return self.fc(cat)

In [406]:
# Instantiate the model w/ hyperparams
vocab_size = len(vocab_to_int)+1 # +1 for the 0 padding + our word tokens
output_size = len(encoded_labels)
filter_size=[2,3,4]
n_filters=100
embedding_dim = 400
hidden_dim = 256
dropout=0.5
n_layers = 2

model = CNN(vocab_size, embedding_dim, n_filters, filter_size, output_size, dropout, pad_idx )

print(model)

NameError: name 'pad_idx' is not defined